In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import KFold
from sklearn import preprocessing

In [2]:
slug = pd.read_csv("220306_slugs_dataset.csv")

In [3]:
slug.head()

,loadCase,slugDen,slugLen,pockDen,pockLen,slugVel,basePer,baseAmp,basePha,simTime,out_oopDisp_mm_n2,out_supReac_kN_n3,out_supReac_kN_n4,out_verDisp_mm_n5,out_oopDisp_mm_n5,out_endBend_kNm_n6
0,00001_sd786_sl6_pd234_pl10_sv6_bp13_ba850_bp74,785.67,6.47,234.16,10.06,5.77,12.55,0.85,74.10,45.8,197.45,16.22,56.42,248.24,180.47,198.49
1,00002_sd705_sl10_pd190_pl11_sv7_bp12_ba800_bp59,705.03,10.36,190.44,11.25,7.42,11.63,0.80,59.11,44.0,186.85,15.97,58.86,262.33,178.72,238.07
2,00003_sd742_sl12_pd159_pl13_sv7_bp12_ba650_bp65,741.98,11.64,159.09,12.52,7.42,11.54,0.65,65.23,48.2,155.47,14.29,48.63,220.89,163.53,180.64
3,00004_sd732_sl9_pd168_pl10_sv10_bp11_ba670_bp68,731.88,8.82,167.88,9.84,9.52,11.48,0.67,67.85,30.8,169.47,15.18,48.72,188.47,163.76,233.31
4,00005_sd669_sl11_pd214_pl9_sv8_bp13_ba810_bp57,669.21,11.46,213.87,8.56,8.18,12.86,0.81,56.68,37.7,223.64,16.90,58.51,271.74,242.37,219.33


In [4]:
slug = slug.sample(frac=1).reset_index(drop=True)

In [5]:
slug_df = slug[["slugDen","slugLen","pockDen","pockLen","slugVel","basePer", "baseAmp", "basePha"]]

In [6]:
scaler = preprocessing.StandardScaler()
scaler = scaler.fit(slug_df)

In [7]:
slug_tar = slug[["out_oopDisp_mm_n2", "out_oopDisp_mm_n5", "out_endBend_kNm_n6"]]

In [8]:
slug_tar.head()

,out_oopDisp_mm_n2,out_oopDisp_mm_n5,out_endBend_kNm_n6
0,255.65,323.52,442.82
1,219.17,255.92,362.34
2,167.29,178.84,189.10
3,212.76,213.70,300.22
4,157.71,149.78,184.17


In [9]:
#train_df = np.array(scaler.transform(slug_df[:4000]))
#train_tar = np.array(slug_tar[:4000])
#test_df = np.array(scaler.transform(slug_df[40000:]))
#test_tar = np.array(slug_tar[40000:])


#slug_train_df = np.array(scaler.transform(slug_df))
#train_tar = np.array(slug_tar[:4000])


kf = KFold(n_splits=5)

In [17]:

#train_df = preprocessing.normalize(train_df)

slug_train_df = preprocessing.normalize(slug_df)

In [99]:
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(256, input_dim=n_inputs, activation='relu'))
    model.add(Dropout(.4))
    model.add(Dense(512, activation='relu'))
    
    #model.add(Dense(1024, activation='relu'))
    
    model.add(Dense(1024, activation='relu'))

    model.add(Dense(1024, activation='relu'))

    model.add(Dense(512, activation='relu'))

    model.add(Dense(512, activation='relu'))
    
    model.add(Dense(n_outputs, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer=RMSprop(lr=0.01, clipnorm=1.), metrics=['mean_squared_error'])
    #model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    return model

In [100]:
#n_inputs, n_outputs = train_df.shape[1], train_tar.shape[1]
n_inputs, n_outputs = slug_df.shape[1], slug_tar.shape[1]

In [101]:
model = get_model(n_inputs, n_outputs)

In [103]:
model.fit(slug_train_df, slug_tar.values, epochs=20, batch_size=2)

In [80]:
for train_index, test_index in kf.split(slug_train_df):
    model.fit(slug_train_df[train_index], slug_tar.iloc[train_index], epochs=20, batch_size=100)

Train on 40000 samples
Epoch 1/20
40000/40000 [==============================] - 1s 35us/sample - loss: nan - mean_squared_error: nan
Epoch 2/20
40000/40000 [==============================] - 1s 23us/sample - loss: nan - mean_squared_error: nan
Epoch 3/20
40000/40000 [==============================] - 1s 23us/sample - loss: nan - mean_squared_error: nan
Epoch 4/20
40000/40000 [==============================] - 1s 23us/sample - loss: nan - mean_squared_error: nan
Epoch 5/20
40000/40000 [==============================] - 1s 23us/sample - loss: nan - mean_squared_error: nan
Epoch 6/20
40000/40000 [==============================] - 1s 23us/sample - loss: nan - mean_squared_error: nan
Epoch 7/20
40000/40000 [==============================] - 1s 23us/sample - loss: nan - mean_squared_error: nan
Epoch 8/20
40000/40000 [==============================] - 1s 23us/sample - loss: nan - mean_squared_error: nan
Epoch 9/20
40000/40000 [==============================] - 1s 23us/sample - loss: nan - me

KeyboardInterrupt: 

In [65]:
test_df[0]

array([ 0.834913  ,  0.1503568 ,  0.81186544,  1.8609707 ,  0.9375468 ,
        1.2187914 ,  0.10141098, -0.46840926])

In [66]:
model.predict(test_df[0:1])

array([[nan, nan, nan]], dtype=float32)

In [67]:
test_tar[0:1]

array([[183.33, 186.68, 220.61]])